In [73]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy




In [74]:
df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)
df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [75]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()+ df_train['dividend'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()+ df_train['dividend'].rolling(window=200, center=False).mean()

df_train = df_train.assign(pma10=pma10)
df_train = df_train.assign(pma20=pma20)
df_train = df_train.assign(pma50=pma50)
df_train = df_train.assign(pma200=pma200)

In [76]:
diffma10 = (df_train['Close']/df_train['pma10'] -1)*100
diffma20 = (df_train['Close']/df_train['pma20'] -1)*100
diffma50 = (df_train['Close']/df_train['pma50'] -1)*100
diffma200 = (df_train['Close']/df_train['pma200'] -1)*100

df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

In [77]:
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)


In [90]:
hh = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hl = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 

df_train = df_train.assign(hh=hh)
df_train = df_train.assign(hl=hl)
df_train = df_train.assign(hh1=hh.shift(-1))
df_train = df_train.assign(hl1=hl.shift(-1))

hh10 = hh.rolling(window=10, center=False).mean()
hl10 = hl.rolling(window=10, center=False).mean()

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

In [84]:
df_train.to_csv("data/spy-train.csv")

In [107]:
print(df_train[['hh','hl']].tail(10))
#print(hl.tail())

               hh     hl
Date                    
2016-03-15  False  False
2016-03-16   True   True
2016-03-17   True   True
2016-03-18   True   True
2016-03-21   True  False
2016-03-22   True  False
2016-03-23  False  False
2016-03-24  False  False
2016-03-28   True   True
2016-03-29   True  False


In [80]:
#print(df_train.columns)

In [81]:
#df_train.describe(include='all')

In [117]:
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend', 
          'pma10','pma20','pma50','pma200',
          #diffma10,diffma20,diffma50,diffma200,
          #pma10tan,pma20tan,pma50tan,pma200tan,
          #'hh-10','hl-10'
          'hh','hl', 'hh1', 'hl1'
         ]
X_train = df_train.drop(remove, axis=1).values[201:-10, :]
X_test = X_train[-10:,:]
X_train = X_train[:-1, :]
#print(X_train)



In [236]:
y_train = hh.shift(-1).values[201:-10].astype(bool)
predict(X_train, y_train, X_test)

y_train = hl.shift(-1).values[201:-10].astype(bool)
predict(X_train, y_train, X_test)


Overall AUC: 0.539655200495
recall: 0.378627968338
precision: 0.599164926931
f1 score: 0.464025869038
[[ 0.47193957  0.52806043]
 [ 0.47922778  0.52077222]
 [ 0.4770505   0.5229495 ]
 [ 0.47562275  0.52437725]
 [ 0.48221529  0.51778471]
 [ 0.47840191  0.52159809]
 [ 0.4882473   0.5117527 ]
 [ 0.48481634  0.51518366]
 [ 0.51157943  0.48842057]
 [ 0.5049012   0.4950988 ]]
Overall AUC: 0.511707026883
recall: 0.335031847134
precision: 0.563169164882
f1 score: 0.420127795527
[[ 0.52478726  0.47521274]
 [ 0.52078817  0.47921183]
 [ 0.51469247  0.48530753]
 [ 0.50781706  0.49218294]
 [ 0.51079455  0.48920545]
 [ 0.49880221  0.50119779]
 [ 0.5090933   0.4909067 ]
 [ 0.49616122  0.50383878]
 [ 0.52422731  0.47577269]
 [ 0.51685876  0.48314124]]


In [231]:
def predict(X_train, y_train, X_test):  
    from sklearn.cross_validation import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC

    model = LogisticRegression(random_state=1)
    #model = RandomForestClassifier(n_estimators=350)
    #model = GradientBoostingClassifier(n_estimators=100)
    #best so far
    #model = KNeighborsClassifier(n_neighbors=15)
    
    # fitting
    X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.25)
    model.fit(X_fit, y_fit);
    y_eval_pred = model.predict_proba(X_eval);
    
    from sklearn.metrics import roc_auc_score
    print('Overall AUC:', roc_auc_score(y_eval,y_eval_pred[:,1]))
    
    from sklearn import metrics
    
    y_eval_comp = y_eval_pred[:,1] > 0.55
    numpy.count_nonzero(y_eval_comp)
    
    print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    y_test_pred = model.predict_proba(X_test);
    print(y_test_pred)
    return



In [227]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    #clf2 = RandomForestClassifier(random_state=1)
    #clf3 = GaussianNB()
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GradientBoostingClassifier(n_estimators=100)
    #clf4 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gb', clf3)], voting='hard')

    for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'Gradient Boosting', 'Ensemble']):
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


    return
